# Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Do some feature engineering by internal DataFrame functions
* Use built-in sklearn functions to compute the regression and access its parameters (coefficients)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

## Import libraries

In [1]:
import sklearn, pandas
import numpy as np

## Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
full_data = pandas.read_csv("/content/drive/MyDrive/FUNIX Progress/MLP302x_1.1-A_EN/data/kc_house_data.csv", index_col=0)

In [3]:
full_data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set your own random seed to ensure consistency.  

In [4]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(full_data, train_size=0.8, test_size=0.2, random_state=0)
train_data = train_data.copy()
test_data = test_data.copy()

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

In [5]:
def extract_features(data, features_title):
    # take the features from the DataFrame
    features = [data[title].values for title in features_title]
    # stack them together for a 2d [examples, properties]
    return np.stack(features, axis=-1)

In [ ]:
# Self-Defined Function

def self_extract_features(data, features_title):
    return data[example_features_title].values

In [13]:
from sklearn.linear_model import LinearRegression

example_features_title = ['sqft_living', 'bedrooms', 'bathrooms']
# extract the features as numpy arrays, and stack them together per each examples
example_features = extract_features(train_data, example_features_title)
example_labels = train_data['price']
example_model = LinearRegression().fit(example_features, example_labels)

Now that we have fitted the model we can extract the regression weights (coefficients) from your model as follows:

In [14]:
example_weight_summary = example_model.coef_
print(example_weight_summary)

[   313.17055038 -56754.66651422   6887.71910816]


## Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing sklearn model's interface functions to analyze multiple regressions. 

Recall that once a model is built we can use the `predict` function to find the predicted values for data we pass. For example using the example model above:

In [17]:
example_predictions = example_model.predict(example_features)
print(round(example_predictions[0], 3)) # should be 395813.499

395813.499


## Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [21]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    prediction = model.predict(data)
    # Then compute the residuals/errors
    residuals = outcome - prediction
    # Then square and add them up
    RSS = sum(residuals**2)

    return(RSS)    

In [18]:
def self_get_residual_sum_of_squares(model, data, outcome):
  return (sum((outcome - model.predict(data))**2))

Test your function by computing the RSS on TEST data for the example model:

In [22]:
example_test_features = extract_features(test_data, example_features_title)
example_test_labels = test_data['price']
rss_example_test = get_residual_sum_of_squares(example_model, example_test_features, example_test_labels)
# rss_example_test = self_get_residual_sum_of_squares(example_model, example_test_features, example_test_labels) 
# 259213572106085.62
print(rss_example_test) # should be ~ 2.5921e+14 or 259213572106085.44

259213572106085.62


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

For now, we will use the default logarithm function of python to create a new feature. You must import it from the math library.

In [23]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 

As an example here's the first one:

In [ ]:
train_data['bedrooms_squared'] = train_data['bedrooms'].map(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].map(lambda x: x**2)

In [ ]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['interaction'] = train_data['bedrooms']*train_data['bathrooms']
test_data['interaction'] = test_data['bedrooms']*test_data['bathrooms']

train_data['log_sqft'] = train_data['sqft_living'].map(lambda x: log(x))
test_data['log_sqft'] = test_data['sqft_living'].map(lambda x: log(x))

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [ ]:
# the pandas.Series have a function called exactly that.
print("Bedroom:", round(test_data['bedrooms_squared'].mean(),2))
print("Bedroom vs. Bathroom:", round(test_data['interaction'].mean(),2))
print("Square Feet:", round(test_data['log_sqft'].mean(),2))
print("Latitude vs. Longtitude:", round(test_data['lat_plus_long'].mean(),2))

Bedroom: 12.21
Bedroom vs. Bathroom: 7.45
Square Feet: 7.55
Latitude vs. Longtitude: -74.65


## Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [ ]:
# ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']

# ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'interaction']
model_2_features = model_1_features + ['interaction']

# ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'interaction', 'bedrooms_squared', 'log_sqft', 'lat_plus_long']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using sklearn's LinearRegression and look at the value of the weights/coefficients:

*Note: regression models in this course often have `coef_`*

In [ ]:
# Learn the three models. Extract the features and fit your models onto respective features.
# If a goldfish is reading, see the example_model above
model_1_features_extracted = extract_features(train_data, model_1_features)
model_2_features_extracted = extract_features(train_data, model_2_features)
model_3_features_extracted = extract_features(train_data, model_3_features)

labels = train_data['price']

model_1 = LinearRegression().fit(model_1_features_extracted, labels)
model_2 = LinearRegression().fit(model_2_features_extracted, labels)
model_3 = LinearRegression().fit(model_3_features_extracted, labels)

In [ ]:
# Examine/extract each model's coefficients. When in doubt, consult the sklearn document.
print('Model 1:', model_1.coef_)
print('Model 2:', model_2.coef_)
print('Model 3:', model_3.coef_)

Model 1: [ 3.12942010e+02 -5.30962691e+04  1.47770428e+04  6.53983343e+05
 -3.25707336e+05]
Model 2: [ 3.06819573e+02 -1.04604718e+05 -7.01815289e+04  6.50590952e+05
 -3.09965751e+05  2.49441497e+04]
Model 3: [ 5.37808087e+02  2.78047781e+03  1.01363765e+05  6.27313968e+12
  6.27313874e+12 -1.81822549e+04  7.24579938e+02 -5.71030023e+05
 -6.27313903e+12]


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

*Answer: Positive*

---

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

*Answer: Negative*

---
Think about what this means.

## Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [ ]:
# Compute the RSS on TRAINING data for each of the three models and display the values. 
# See the prior lab if you have already forgotten how.
model_1_rss = get_residual_sum_of_squares(model_1, model_1_features_extracted, labels)
model_2_rss = get_residual_sum_of_squares(model_2, model_2_features_extracted, labels)
model_3_rss = get_residual_sum_of_squares(model_3, model_3_features_extracted, labels)

print(model_1_rss)
print(model_2_rss)
print(model_3_rss)

979843597588334.5
970799199729578.5
913653656189829.5


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

*Answer: Model 3*

Now compute the RSS on on TEST data for each of the three models.

In [ ]:
# Compute the RSS on TESTING data for each of the three models and display the values:
model_1_test_features = extract_features(test_data, model_1_features)
model_2_test_features = extract_features(test_data, model_2_features)
model_3_test_features = extract_features(test_data, model_3_features)

test_labels = test_data['price']

model_1_test_rss = get_residual_sum_of_squares(model_1, model_1_test_features, test_labels)
model_2_test_rss = get_residual_sum_of_squares(model_2, model_2_test_features, test_labels)
model_3_test_rss = get_residual_sum_of_squares(model_3, model_3_test_features, test_labels)

print(model_1_test_rss)
print(model_2_test_rss)
print(model_3_test_rss)

213487129319103.72
210778544168942.22
203972399256564.47


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

*Answer: Model 3*